In [8]:
import pandas as pd
import numpy as np
import pickle

In [6]:
pickles_path = "../../../pickles/"

In [7]:
# unpickle
with open(pickles_path + "en_1218_traffic_summary_by_type.pkl", 'rb') as picklefile: 
    traffic_summary_by_type = pickle.load(picklefile)

traffic_summary_by_type

,edges_count,traffic_volume,edges_count_pct,traffic_volume_pct,traffic_avg
type,,,,,
external,10990982,5100299091,0.368235,0.741460,464.043985
link,17851574,1718399222,0.598088,0.249814,96.260376
other,1005180,60022438,0.033677,0.008726,59.713124


In [9]:
# unpickle
with open(pickles_path + "en_1218_external_traffic_summary_by_cat.pkl", 'rb') as picklefile: 
    external_traffic_summary_by_cat = pickle.load(picklefile)

external_traffic_summary_by_cat

,edges_count,traffic_volume,edges_count_pct,traffic_volume_pct,traffic_avg
prev,,,,,
other-empty,5093433,1779965188,0.463419,0.348992,349.462767
other-external,787311,116732857,0.071632,0.022887,148.267784
other-internal,1348892,132349844,0.122727,0.025949,98.117450
other-other,374921,36972380,0.034112,0.007249,98.613788
other-search,3386425,3034278822,0.308109,0.594922,896.012409


From the [EDA notebook](English_Wikipedia_EDA.ipynb#Distribution-of-traffic-volume-and-edge-counts-across-the-3-reference-types):  

number of terminal nodes in link edges = 2598302  
number of terminal nodes in internal search edges = 414847

### Data for the pie/donut chart segments  

In [61]:
traffic_by_type = external_traffic_summary_by_cat[["traffic_volume", "edges_count"]].copy()
traffic_by_type.sort_values("traffic_volume", ascending=False, inplace=True)

temp = traffic_summary_by_type[["traffic_volume", "edges_count"]][traffic_summary_by_type.index != "external"].copy()

traffic_by_type = traffic_by_type.append(temp)
traffic_by_type.columns = ["traffic", "terminal_articles_count"]
traffic_by_type.index.name = "traffic_type"

traffic_by_type.loc["link", "terminal_articles_count"] = 2598302
traffic_by_type.loc["other", "terminal_articles_count"] = 414847

traffic_by_type

,traffic,terminal_articles_count
traffic_type,,
other-search,3034278822,3386425
other-empty,1779965188,5093433
other-internal,132349844,1348892
other-external,116732857,787311
other-other,36972380,374921
link,1718399222,2598302
other,60022438,414847


In [62]:
total_traffic = traffic_by_type.traffic.sum()
total_traffic

6878720751

In [63]:
traffic_by_type["traffic_pct"] = round(traffic_by_type.traffic / total_traffic * 100, 2)
traffic_by_type

,traffic,terminal_articles_count,traffic_pct
traffic_type,,,
other-search,3034278822,3386425,44.11
other-empty,1779965188,5093433,25.88
other-internal,132349844,1348892,1.92
other-external,116732857,787311,1.70
other-other,36972380,374921,0.54
link,1718399222,2598302,24.98
other,60022438,414847,0.87


From the [EDA notebook](English_Wikipedia_EDA.ipynb#How-many-articles-are-there-in-the-dataset?):  

total number of articles in the dataset = 5185699

In [64]:
total_articles_count = 5185699

In [65]:
traffic_by_type["terminal_articles_pct"] = round(traffic_by_type.terminal_articles_count 
                                                 / total_articles_count * 100, 2)

In [66]:
# add display labels for viz

traffic_by_type.loc["other-empty", "label"] = "empty referer"
traffic_by_type.loc["other-external", "label"] = "external websites"
traffic_by_type.loc["other-internal", "label"] = "other Wikimedia"
traffic_by_type.loc["other-other", "label"] = "unknown"
traffic_by_type.loc["other-search", "label"] = "search engines"
traffic_by_type.loc["link", "label"] = "links"
traffic_by_type.loc["other", "label"] = "internal search"

In [67]:
traffic_by_type

,traffic,terminal_articles_count,traffic_pct,terminal_articles_pct,label
traffic_type,,,,,
other-search,3034278822,3386425,44.11,65.30,search engines
other-empty,1779965188,5093433,25.88,98.22,empty referer
other-internal,132349844,1348892,1.92,26.01,other Wikimedia
other-external,116732857,787311,1.70,15.18,external websites
other-other,36972380,374921,0.54,7.23,unknown
link,1718399222,2598302,24.98,50.11,links
other,60022438,414847,0.87,8.00,internal search


In [68]:
len(traffic_by_type)

7

In [69]:
traffic_by_type.to_csv("traffic_by_type.csv")

In [70]:
!head traffic_by_type.csv

traffic_type,traffic,terminal_articles_count,traffic_pct,terminal_articles_pct,label
other-search,3034278822,3386425,44.11,65.3,search engines
other-empty,1779965188,5093433,25.88,98.22,empty referer
other-internal,132349844,1348892,1.92,26.01,other Wikimedia
other-external,116732857,787311,1.7,15.18,external websites
other-other,36972380,374921,0.54,7.23,unknown
link,1718399222,2598302,24.98,50.11,links
other,60022438,414847,0.87,8.0,internal search
